In [147]:
import numpy as np
import pandas as pd
import scipy
import os
import librosa
import librosa.display

In [148]:
# Extract Raw Data Files
fs = 48000
path = "/Users/ares/PycharmProjects/pythonProject/Emotion_Classifier/data/"
os.chdir(path)
filename = os.listdir(path)

In [149]:
raw = []
length = []
label = []
for wavfile in filename:
    emotion = ''.join([i for i in wavfile[:-7]])
    #index = int(wavfile[-7:-4])
    waveform, rate = librosa.load(path + wavfile, sr = fs)
    waveform = waveform.astype('float64')
    effective = waveform[np.argwhere(waveform/max(waveform) > 0.05)[0,0]:np.argwhere(waveform/max(waveform) > 0.05)[-1,0]+1]
    raw.append(effective)
    length.append(len(effective))
    label.append(emotion)

In [150]:
label = np.array(label)
raw = np.array(raw)

/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/2823102939.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  raw = np.array(raw)


In [194]:
feature = ['emotion','flatness','zerocr','meanMag','maxMag','meancent','stdcent','maxcent','stdMag'] + \
          ['pitchmean','pitchmax','pitchstd','pitch_tuning_offset','meanrms','maxrms','stdrms'] + \
          ['mfcc_' + str(i+1) + '_mean' for i in range(50)] + \
          ['mfcc_' + str(i+1) + '_std' for i in range(50)] + \
          ['mfcc_' + str(i+1) + '_max' for i in range(50)] + \
          ['chroma' + str(i+1) + '_mean' for i in range(12)] + \
          ['mels' + str(i+1) + '_mean' for i in range(128)] + \
          ['contrast' + str(i+1) + '_mean' for i in range(7)]

feature_pd = pd.DataFrame(columns=feature)

In [195]:
for i in range (1125):
    saving = []
    saving.append(label[i])
    stft = np.abs(librosa.stft(raw[i]))
    #flatness
    flatness = np.mean(librosa.feature.spectral_flatness(y=raw[i]))
    saving.append(flatness)
    #zero-crossing
    zerocr = np.mean(librosa.feature.zero_crossing_rate(raw[i]))
    saving.append(zerocr)

    S, phase = librosa.magphase(stft)
    meanMagnitude = np.mean(S)
    stdMagnitude = np.std(S)
    maxMagnitude = np.max(S)
    saving.append(meanMagnitude)
    saving.append(maxMagnitude)

    #Centroid
    cent = librosa.feature.spectral_centroid(y=raw[i], sr=fs)
    cent = cent / np.sum(cent)
    meancent = np.mean(cent)
    stdcent = np.std(cent)
    maxcent = np.max(cent)
    saving.append(meancent)
    saving.append(stdcent)
    saving.append(maxcent)
    saving.append(stdMagnitude)

    # pitch
    pitches, magnitudes = librosa.piptrack(y=raw[i], sr=fs, S=stft, fmin=70, fmax=400)
    pitch = []
    for j in range(magnitudes.shape[1]):
        indexs = magnitudes[:, 1].argmax()
        pitch.append(pitches[indexs, j])
    pitch_tuning_offset = librosa.pitch_tuning(pitches)
    pitchmean = np.mean(pitch)
    pitchstd = np.std(pitch)
    pitchmax = np.max(pitch)
    pitchmin = np.min(pitch)
    saving.append(pitchmean)
    saving.append(pitchmax)
    saving.append(pitchstd)
    saving.append(pitch_tuning_offset)

    #rmse
    rmse = librosa.feature.rms(S=S)[0]
    meanrms = np.mean(rmse)
    stdrms = np.std(rmse)
    maxrms = np.max(rmse)
    saving.append(meanrms)
    saving.append(maxrms)
    saving.append(stdrms)

    #mfcc
    mfccs = np.mean(librosa.feature.mfcc(y=raw[i], sr=fs, n_mfcc=50).T, axis=0)
    mfccsstd = np.std(librosa.feature.mfcc(y=raw[i], sr=fs, n_mfcc=50).T, axis=0)
    mfccmax = np.max(librosa.feature.mfcc(y=raw[i], sr=fs, n_mfcc=50).T, axis=0)
    saving.extend(mfccs.tolist())
    saving.extend(mfccsstd.tolist())
    saving.extend(mfccmax.tolist())

    #chromagram
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=fs).T, axis=0)
    saving.extend(chroma.tolist())

    #mels
    mel = np.mean(librosa.feature.melspectrogram(y=raw[i], sr=fs).T, axis=0)
    saving.extend(mel.tolist())
    #ottava对比
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=fs).T, axis=0)
    saving.extend(contrast.tolist())
    print(i)
    feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)

0
1
2


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


3
4
5


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


6
7
8
9


...

/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


1119
1120
1121
1122


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipy

1123
1124


/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)
/var/folders/dn/j0_7h7gx2sq58_nt9816ss480000gn/T/ipykernel_6547/4250457866.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  feature_pd = feature_pd.append(pd.DataFrame(saving, index=feature).transpose(), ignore_index=True)


In [196]:
feature_pd

,emotion,flatness,zerocr,meanMag,maxMag,meancent,stdcent,maxcent,stdMag,pitchmean,...,mels126_mean,mels127_mean,mels128_mean,contrast1_mean,contrast2_mean,contrast3_mean,contrast4_mean,contrast5_mean,contrast6_mean,contrast7_mean
0,calm,0.025215,0.060741,0.026846,9.410904,0.00885,0.008068,0.029945,0.161053,0.0,...,0.0,0.0,0.0,17.84417,12.688804,15.974683,16.461388,18.016168,21.397066,21.430682
1,sad,0.007192,0.107056,0.09987,22.972229,0.007353,0.00536,0.019579,0.511499,66.514207,...,0.000001,0.0,0.0,11.807734,22.546581,20.587265,20.014085,20.327124,22.80111,29.031935
2,sad,0.00887,0.059829,0.093057,37.875198,0.005348,0.004205,0.017702,0.619473,44.179797,...,0.000001,0.0,0.0,20.393315,12.639328,15.273654,16.208149,17.552429,20.433561,24.568633
3,calm,0.011513,0.058887,0.024574,8.392821,0.006667,0.004371,0.019219,0.174096,44.710801,...,0.0,0.0,0.0,19.37327,17.798835,18.696435,15.987231,18.742845,18.05437,24.122616
4,fearful,0.007202,0.056484,0.444939,110.891922,0.006289,0.004181,0.017703,2.296544,26.856039,...,0.000094,0.000008,0.0,21.966236,14.973455,14.80832,16.422721,18.234495,17.717152,30.875616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,sad,0.007115,0.049032,0.047011,12.353555,0.006944,0.004845,0.024145,0.336476,44.090273,...,0.0,0.0,0.0,16.604176,18.391297,21.093146,19.641472,21.158854,21.537565,25.479859
1121,calm,0.004772,0.04181,0.03154,8.660782,0.007299,0.005194,0.020839,0.204246,102.420125,...,0.0,0.0,0.0,17.410227,12.587538,16.747762,16.252896,17.609586,19.764173,23.710072
1122,disgust,0.017978,0.074625,0.078325,26.815812,0.005435,0.00328,0.015725,0.432133,30.942438,...,0.000003,0.0,0.0,18.45115,13.566328,15.898536,15.068925,18.513952,17.315592,27.457853
1123,angry,0.009161,0.087952,0.114873,20.943423,0.006579,0.00612,0.022281,0.597547,53.742613,...,0.000005,0.0,0.0,16.184287,19.305839,19.719753,19.085318,20.619816,20.788578,27.559495


In [197]:
feature_pd.to_csv('mfcc_mean_caca.csv', index=False)